<center><font size = "8">Passive and Active membrane Properties<center>

<font size=4><font color=blue> In this notebook you will learn:
<ul style="color: blue; font-size: 18px;">
    <li> the differences between passive and active membrane properties and how this affects the signal propagation along the dendrites</li>
</ul>

<p style="color: red; font-size: 16px;"><b>IMPORTANT:</b> this notebook is using <b>Python</b> code to help you load, visualize and analyze the morphologies.</p>
<ul style="color: red; font-size: 16px;">
  <li><b>It is not necessary to know how to code</b> (but it would be good for you if you try to understand the code)</li>
  <li>To only run the cells: </li>
    <ul style="color: red; font-size: 16px;">
        <li>Click once on one of the cells, you will notice that it is sellected because it is highlighted in blue.</li>
        <li>Press "Command+enter" or click on the Run (triangle) button at the top of the page.</li>
        <li>Consider that cells <b>must be run by order</b> </li>
    </ul>
</ul>

<p style="color: green; font-size: 16px;"><b>YOUR GOAL:</b></p>
<ul style="color: green; font-size: 16px;">
  <li>Answer the questions that you would find along the notebook.</li>
  <li>For that you would have to understand what are the passive and the active properties of a neuron and how they affect signal propagation</li> 
  <li>  Build a report with your answeres and conclussions.</li>
</ul>

In [ ]:
# This cell will load some useful files into your notebook environment

import requests, os

extra_files_root = "https://raw.githubusercontent.com/NataliBZ/uc3m_notebooks/main/"

extra_files_names = [
    "Cellular/02_Passive_Active_properties/TTXDynamicsSwitch.mod",
    "Cellular/02_Passive_Active_properties/UsefulFunctions.py",
    "Cellular/02_Passive_Active_properties/active_neuron_1d_3d.py",
    "Cellular/02_Passive_Active_properties/cell_01.swc",
    "Cellular/02_Passive_Active_properties/cell_02.swc",
    "Cellular/02_Passive_Active_properties/equivalent_circuit.png",
    "Cellular/02_Passive_Active_properties/hoc2swc.py",
    "Cellular/02_Passive_Active_properties/passive_neuron_1d_3d.py",
    "Cellular/02_Passive_Active_properties/passive_neuron_1dend.py"
]
for fn in extra_files_names:
    req = requests.get(extra_files_root + fn)
    if req.status_code == 200:
        with open(os.path.split(fn)[1], "wb") as fid:
            fid.write(req.content)
    else:
        print("Error obtaining auxiliary file!")

## 1. Passive and active membrane properties

<font size="4">
    
__What are passive and active membrane properties?:__ 

- __Passive__ membrane properties are characteristics of a neuronal membrane, such as membrane resistance, membrane capacitance, and axial resistance, that do not depend on voltage-gated ion channels or receptors but are instead related to ion leakage channels, pumps, and the cell's physical structure. These properties influence how the membrane conducts electrical signals by affecting the efficiency of signal propagation, the speed of potential changes, and the integration of synaptic inputs. (You can know more about passive properties in the following [link](https://www.scientifica.uk.com/learning-zone/passive-membrane-properties))

- __Active__ membrane properties depend on the activation of voltage gated ion channels. Unlike passive properties, active properties allow the neuron to generate and regulate action potentials, amplify or dampen inputs, and produce nonlinear responses.

- This can be represented (modeled) as an RC circuit, with resistance and capacitance connected in parallel. Passive properties involve components that store, dissipate, or filter energy, such as resistors, capacitors, and inductors, and do not require an external power source to operate. For example, the lipid bylayer of the membrane acts as a capacitance with channels that allow the ions to pass through with certain resistance (membrane resistivity). In case of axons and dendrites, the intracelular axial resistance is a passive membrane property. This property leads to decay of the membrane potential in a distance dependent manner, starting from  the stimulation point. In contrast, active properties are associated with components that require an external power supply to amplify, generate, or control energy in the form of voltage or current, with examples including voltage/current sources and transistors. 

<center><img src="equivalent_circuit.png" width="500"><center>

### Let's start!

In [ ]:
# First call all the usefull packages
%matplotlib inline

from neuron import h
import matplotlib.pyplot as plt
import numpy as np
import math as m
import neurom as nm
from neurom import view
from hoc2swc import neuron2swc
import UsefulFunctions as UF

import ipywidgets as widgets
from IPython.display import display, clear_output

from passive_neuron_1dend import PassiveNeuron_1Dend
from passive_neuron_1d_3d import PassiveNeuron_1D_3D
from active_neuron_1d_3d import ActiveNeuron_1D_3D

## 1.1 Signal attenuation due to passive membrane properties: 
## using a toy neuronal model: soma, 1 dendrite and 1 axon

In [ ]:
cell_pass_1d = PassiveNeuron_1Dend()

# Plot morphology so far
UF.plot_morphology("cell_01")

<font size=4> In this very simple and NON REALISTIC model:

<ul style="color: black; font-size: 18px;">
    <li> the soma is represented as a black square of 20x20 µm</li>
    <li> the axon (blue line) is defined by the following parameters: 
        <ul><li> diam = 3 µm </li></ul>
        <ul><li> length = 100 µm </li></ul>
        <ul><li> axial resistance = 123.0 Ω*cm </li></ul>
        <ul><li> capacitance = 1 µF/cm^2 </li></ul>
    <li> the dendrite (red line) is defined by the following parameters:</li>
        <ul><li> diam = 1 µm </li></ul>
        <ul><li> length = 200 µm </li></ul>
        <ul><li> axial resistance = 300.0 Ω*cm </li></ul>
        <ul><li> capacitance = 1 µF/cm^2 </li></ul>
</ul>  

<font size=3>*Note: if you are curious about how this neurons has been coded in python, check the file "passive_neuron_1dend.py"

### Now we are going to see how the signal propagates into the soma from three different stimulation points in the dendrite.

In [ ]:
UF.reset()

# Scan at different locations with different delays
delays = np.arange(100, 600, 200)
locations = np.linspace(0, 1, 3)

# Setup stimulations and recordings
for p in zip(locations, delays):
    UF.iclamp(cell_pass_1d.dend(p[0]), amplitude=0.4, delay=p[1], duration=50)
UF.record_voltage(cell_pass_1d.soma(0.5))

# Simulate and plot
v_init = -70
t_stop = 700
UF.tvi_plots(UF.init_run(v_init, t_stop), UF.voltage_records, UF.current_records, vmax=0)

<font size=4>You can see that the signal recording at the soma (voltages) is larger for the dendritic stimulation closer to it (dend(0)), while is smaller for the fardest dendritic stimulation (dend(1)). Just due to the physical properties of the neuron.

### Let's see what would happen if we stimulate at the soma and record at different points the dendrite

In [ ]:
UF.reset()

# Scan at different locations with different delays
locations = np.linspace(0, 1, 3)

# Setup stimulations and recordings
UF.iclamp(cell_pass_1d.soma(0.5), amplitude=0.4)
for p in locations:
    UF.record_voltage(cell_pass_1d.dend(p))

# Simulate and plot
v_init = -70
t_stop = 700
UF.tvi_plots(UF.init_run(v_init, t_stop), UF.voltage_records, UF.current_records, vmax=-10)

## INTERACTIVE:
### Play around with the following passive parameters and observe how the voltage changes along the dendrite:
    - dendritic diamater
    - dendritic axial resistance
    - dendritic capacitance

In [ ]:
UF.chage_passive_prop(cell_pass_1d)

<font size=6><font color=blue> Question 1:

<font size=4><font color=blue>  After exploring different values for the passive properties such as dendritic diameter, axial resistance and dendritic capacitance, explain what is the relationship between the signal propagation and each of the explored passive properties.

<font size=4><font color=blue>As observed, stimulating the dendrite at points closer to the soma (e.g., dend(0)) results in a larger voltage signal recorded at the soma than stimulation from points farther away (e.g., dend(1)). Explain the biophysical properties of the neuron that account for this decrease in signal strength with increasing distance.

## 1.2 Signal attenuation due to passive membrane properties: 
### using a toy neuronal model: soma, dendrite with 4 dendritic branches with different diametres and 1 axon

In [ ]:
cell_pass_1d_3d = PassiveNeuron_1D_3D()

# Plot morphology so far
UF.plot_morphology("cell_02")

### As we did before we will stimulate at the final end of each dendrite and record the voltage signal at the soma

In [ ]:
UF.reset()

branchlst = [cell_pass_1d_3d.branch_0, cell_pass_1d_3d.branch_1, cell_pass_1d_3d.branch_2, cell_pass_1d_3d.branch_3]

# Setup stimulations and recordings
for idx, branch in enumerate(branchlst):
    UF.iclamp(branch(0.5), amplitude=0.4, delay=200 + idx * 100, duration=50)
UF.record_voltage(cell_pass_1d_3d.soma(0.5))

# Simulate and plot
v_init = -70
t_stop = 700
UF.tvi_plots(UF.init_run(v_init, t_stop), UF.voltage_records, UF.current_records, vmax=-65)

### As before we will stimulate at the soma and record at the end of each dendrite

In [ ]:
UF.reset()

# Setup stimulations and recordings
UF.iclamp(cell_pass_1d_3d.soma(0.5), amplitude=0.4)
for branch in branchlst:
    UF.record_voltage(branch(1))

# Simulate and plot
v_init = -70
t_stop = 700
UF.tvi_plots(UF.init_run(v_init, t_stop), UF.voltage_records, UF.current_records, vmax=-66)

<font size=6><font color=blue> Question 2:

<font size=4><font color=blue>  Based on your answer to question 1 regarding dendritic diameter, how would you explain what is happening to the signal propagation into the soma or through the dendritic branches in the previous example of point 1.2

<font size=4><font color=blue>Based on the parameters modeled from both neuronal structures built, why is the axial resistance higher in the dendrite than in the axon? Which specific biophysical factors are behind this difference? (Hint: check how the models are built in files "passive_neuron_1dend.py" and "passive_neuron_1d_3d.py")

## 1.3 Active properties.  
## We will perform the same experiments but including active properties at the soma and the dendrites

In [ ]:
# Firts we load the neuron with active properties:

cell_active_1d_3d = ActiveNeuron_1D_3D()

# Plot morphology so far
UF.plot_morphology("cell_02")

# Back-propagating action potential.
<p style="font-size: 16px;">
Normally, when a neuron fires an action potential, it starts at the axon hillock (near the cell body) and travels forward along the axon to send signals to other cells, that is called forward propagation.
</p>
<p style="font-size: 16px;">
At the same time, a copy of that electrical signal also travels backward, from the soma (cell body) into the dendrites. That is called a back-propagating action potential (bAP).
</p>

<p style="font-size: 16px;"><b>Why does it happen?</b></p>
<p style="font-size: 16px;">
Because dendrites are made of excitable membrane too, they have voltage-gated sodium and potassium channels, just like the axon.
So when the action potential starts, some of that depolarization can spread backward and briefly activate those channels in the dendrites.
</p>

<p style="font-size: 16px;"><b>Does it has a functionality?</b></p>

The bAP does not send a message to other neurons, but it’s very important inside the neuron itself:
<ul style="font-size: 16px;">
        <li>It tells the dendrites that the cell has fired — kind of a “heads-up” signal.</li>
        <li>It helps in synaptic plasticity (learning): when a bAP coincides with input at a synapse, that connection can get stronger or weaker (this is part of spike- timing–dependent plasticity).</li>
        <li>It can help regulate ion channel distribution and gene expression in dendrites.</li>
</ul>

In [ ]:
UF.reset()

branchlst = [cell_active_1d_3d.branch_0, cell_active_1d_3d.branch_1, cell_active_1d_3d.branch_2, cell_active_1d_3d.branch_3]

# Setup stimulations and recordings
for branch in branchlst:
    UF.iclamp(branch(1), amplitude=0.5, delay=100, duration=50)

UF.record_voltage(cell_active_1d_3d.soma(0.5))
UF.record_voltage(cell_active_1d_3d.dend(0.5))

# Simulate and plot
v_init = -70
t_stop = 700
UF.tvi_plots(UF.init_run(v_init, t_stop), UF.voltage_records, UF.current_records, vmax=60, imax=1)

<font size=6><font color=blue> Question 3:

<font size=4><font color=blue>  Imagine that you are studying how the signal propagates along two different neurons, one with dendrites two times ticker than the other. Considering only the passive properties, use what you have learnt during the lecture and this practical session to hypothesised which neuron would be easier to activate under the same stimulation.

<font size=6><font color=blue> Question 4:

<font size=4><font color=blue>  Imagine now that the neuron with thicker dendrites also has four times higher active membrane conductance (i.e., due to ion channels), which neuron would be easier to activate under the same stimulation, and why?

<ul style="color: green; font-size: 24px;">
  If this notebook sparked your interest about studying single cell behavior, feel free to explore the data and build your own single cell model on the Open Brain Platform.
</ul>